---
draft: true
title: Finding pole positions
---

In [ ]:
# | code-fold: true
# | code-summary: Import Python libraries
from __future__ import annotations

from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated
from iminuit import Minuit
from IPython.display import Math

In [ ]:
# | echo: false
%config InlineBackend.figure_format = 'svg'

## Theory

Define $T$ matrix via $K$-matrix:
$$T(s) = \left(\mathbb{I} + i K(s) {\rho_{cm}(s)}\right)^{-1} K(s)$$
For one channel and multiple resonances this gives:\
$$\sum_\mathrm{Res.}\frac{{K(s)}}{i {K(s)} {{\rho_{cm}(s)}} + 1}$$

A resonance pole in the complex $s$ plane occurs when the denominator of $T(s)$ goes to zero:
$$|T(s)| \rightarrow \infty \quad \Rightarrow \quad \left| \frac{1}{1 - i K(s)} \right|^2 \rightarrow \infty$$

Finding the poles is equivalent to minimizing:

$$\left|1 - i K(s)\right|^2 \rightarrow 0$$

to solve the problem numerically one can separate real and complex part of $s$ to construct a cost function which can be given to common minimizer: Minuit2
Cost function:
$$\text{Cost}(x, y) = \left|1 - i K(x + i y)\right|^2$$
using where $x$ and $y$ are minimization parameters.

## Symbolic implementation

In [ ]:
# | code-fold: true
# | code-summary: Definition of phase space factors
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


@unevaluated(real=False)
class PhaseSpaceFactorCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))

In [ ]:
n = 1
K = sp.MatrixSymbol("K", n, n)
rho_cm = sp.MatrixSymbol(R"\rho^\mathrm{CM}", n, n)
rho = sp.MatrixSymbol("rho", n, n)
I = sp.Identity(n)

In [ ]:
T1 = (I - sp.I * rho_cm * K).inv() * K
T1

In [ ]:
T1.as_explicit()

In [ ]:
s, g1, m1, g2, m2 = sp.symbols("s g1 m1 g2 m2")
ma, mb = sp.symbols("m_a m_b")

In [ ]:
definitions = {
    K[0, 0]: g1**2 / (m1**2 - s) + g2**2 / (m2**2 - s),
    rho_cm[0, 0]: PhaseSpaceFactorCM(s, ma, mb),
    rho[0, 0]: PhaseSpaceFactor(s, ma, mb),
}
Math(aslatex({k: v.doit(deep=False) for k, v in definitions.items()}))

In [ ]:
T1_expr = T1.as_explicit().subs(definitions)[0].simplify(doit=False)
T1_expr

In [ ]:
T2 = (T1.inv() - 2 * sp.I * rho).inv()
T2

In [ ]:
T2_expr = T2.as_explicit().subs(definitions)[0].simplify(doit=False)
T2_expr

In [ ]:
parameters = {
    m1: 1.8,
    m2: 1.1,
    g1: 0.5,
    g2: 0.7,
    ma: 0.1,
    mb: 0.2,
}

In [ ]:
T1_func = sp.lambdify(s, T1_expr.doit().subs(parameters))
T2_func = sp.lambdify(s, T2_expr.doit().subs(parameters))

In [ ]:
# | code-fold: true
x = np.linspace(0.1, 6, 500)
z = T2_func(x - 1e-8j)
plt.plot(x, np.abs(z) ** 2)
plt.xlabel("s")
plt.ylabel(R"$|T(s)|^2$")
plt.yticks([])
plt.ylim(0, None)
plt.show()

In [ ]:
# | code-fold: true
X, Y = np.meshgrid(
    np.linspace(0, 6, num=500),
    np.linspace(1e-8, 2, num=500),
)
S = X + 1j * Y
style = dict(rasterized=True, vmin=-1, vmax=1)
plt.pcolormesh(X, +Y, T1_func(S).imag, **style)
plt.pcolormesh(X, -Y, T2_func(S.conj()).imag, **style)
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.colorbar()
plt.show()

In [ ]:
numerator, denominator = sp.fraction(T2_expr)
denominator

In [ ]:
denom_func = sp.lambdify(s, denominator.doit().subs(parameters))

In [ ]:
# | code-fold: true
plt.pcolormesh(X, -Y, np.abs(denom_func(S.conj())), rasterized=True, vmin=0, vmax=3)
plt.colorbar()
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.show()

In [ ]:
s_guess1 = 1.8 - 0.26j
s_guess2 = 3.7 - 0.47j

## Find pole positions

In [ ]:
def fit_pole(s_guess: complex) -> Minuit:
    minuit2 = Minuit(cost_function, s_guess.real, s_guess.imag)
    minuit2.tol = 0.001
    return minuit2.migrad()


def cost_function(s_real: float, s_imag: float):
    s = s_real + s_imag * 1j
    return np.abs(denom_func(s)) ** 2

In [ ]:
fit_pole1 = fit_pole(s_guess1)
pole1 = complex(*fit_pole1.values)
pole1

In [ ]:
fit_pole2 = fit_pole(s_guess2)
pole2 = complex(*fit_pole2.values)
pole2

In [ ]:
# | code-fold: true
plt.pcolormesh(X, +Y, T1_func(S).imag, **style)
plt.pcolormesh(X, -Y, T2_func(S.conj()).imag, **style)
plt.plot(pole1.real, pole1.imag, "rx", markersize=10)
plt.plot(pole2.real, pole2.imag, "rx", markersize=10)
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.colorbar()
plt.show()

## Compute residues

$$
\begin{align}
\mathrm{Res}(T, z_0)
    &= \frac{1}{2\pi i} \oint T(z) \, dz \\
    &= \frac{1}{2\pi i} \int_{-\pi}^{+\pi} T(z) \; \underbrace{i r e^{i\phi}}_{dz/d\phi} \; d\phi
    \quad \text{with} \quad z = z_0 + r e^{i\phi} \\
    &= \frac{r}{2\pi} \int_{-\pi}^{+\pi} T(z) \, e^{i\phi} \, d\phi \\
    &= \frac{r}{2\pi} \left(\frac{2\pi}{N}\sum_{i=1}^N T(z_i) \, e^{i\phi_i}\right) \\
    &= \frac{r}{N} \sum_{i=1}^N T(z_i) \, e^{i\phi_i}
\end{align}
$$

In [ ]:
def compute_residue(f, z0, radius=1e-3, n_points=1_000):
    phi = np.linspace(-np.pi, np.pi, n_points, endpoint=False)
    z = z0 + radius * np.exp(1j * phi)
    return radius / n_points * np.sum(f(z) * np.exp(1j * phi))

In [ ]:
residue1 = compute_residue(T2_func, pole1)
residue1.round(3)

In [ ]:
residue2 = compute_residue(T2_func, pole2)
residue2.round(3)

Result can be checked with $\lim\limits_{z \to z_0} T(z) = \frac{\mathrm{Res}(T, z_0)}{z - z_0}$, for instance by ploting $T(z)$ over $z=z_0+re^{i\phi}$ with $\phi \in [-\pi, +\pi]$.

In [ ]:
# | code-fold: true
phi = np.linspace(-np.pi, np.pi, num=1000)
r = 0.1
z = pole1 + r * np.exp(1j * phi)
plt.plot(phi, T2_func(z).imag, ls="--")
plt.plot(phi, (residue1 / (z - pole1)).imag, ls="--")
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\mathrm{Im}\,T(z)$")
plt.show()